### ELN WRITEUP COMPARISON BETWEEN DM INSTANCES:

- run each 'cell' one by one; once you get to [Cell 3](#cell-3):
- simply select the CRO protocol name and click 'Submit' button
    - it will read the list of experiment ids from the text file
- afterwards when you reach [Cell 4](#cell-4) an html render of those specific experiment ids will be executed
- check the valid/invalid textboxes as you are QC'ing the writeup data
- when finished, run [Cell 5](#cell-5) and a blue button will appear; clikc this to save the checkboxes
- lastly, execute [Cell 6](#cell-6) in order to export the saved experiment ids and their `validity` status as a `.csv` file

## Cell 1

In [ ]:
from os import getenv
from dotenv import load_dotenv
load_dotenv()
DB_USER = getenv("DB_USER")
DB_PASS = getenv("DB_PASS")
DB_HOST = getenv("DB_HOST")
DB_PORT = getenv("DB_PORT")
DB_NAME = getenv("DB_NAME")

## Cell 2

In [ ]:
%run compare_eln_writeup_scrape.ipynb

## Cell 3

In [ ]:
dropdown = widgets.Dropdown(
    options=cro_prots,
    value=cro_prots[0],
    description="Select:",
    style={"description_width": "initial"},
)

button = widgets.Button(description="Submit")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print(dropdown.value)

button.on_click(on_button_click)
display(dropdown, button, output)

##### Only run [Cell 4](#cell-4) after executing each cell above and selecting the cro protocol then click 'Submit'

## Cell 4

In [ ]:
idx = 0
dd_value = dropdown.value
print ('='*15, f'{dd_value}', '='*15)
html_output = ""
exp_ids = []
with open(os.path.join(dir_path, file_name_prefix.format(dd_value)), 'r') as file:
    for line in file:
        exp_ids.extend(line.strip().split(","))
        
filtered_df = df[df['exp_id'].isin(exp_ids)].copy()
filtered_df['write_up'] = filtered_df.apply(color_code_writeup, axis=1) 

for exp_id, group in filtered_df.groupby('exp_id'):
    idx += 1
    html_output += render_html_group(idx, group)
html_output += js_code
display(HTML(html_output))

##### Run [Cell 5](#cell-5) and click the 'Update Valid Exp Ids' blue button

## Cell 5

In [ ]:
%%javascript

const div = document.createElement('div');
div.innerHTML = '<p style="margin-top: 10px; font-size: 20px; color: gray;">Click the button only after having finished validating each experiment ID.</p>';
element.append(div);

const updateButton = document.createElement('button');
updateButton.textContent = 'Save Valid Exp Ids';
updateButton.style.padding = '12px 24px';
updateButton.style.fontSize = '18px';
updateButton.style.borderRadius = '8px';
updateButton.style.backgroundColor = '#007BFF';
updateButton.style.color = 'white';
updateButton.style.border = 'none';
updateButton.style.cursor = 'pointer';
updateButton.style.marginTop = '10px';
updateButton.style.transition = 'transform 0.3s ease'
element.append(updateButton);
updateButton.addEventListener('click', function() {
    postValidityBackend();
    div.innerHTML = '<p style="margin-top: 10px; font-size: 20px; color: gray;">Updated validity data (run the following cell to view & export to file)</p>';
    updateButton.style.animation = 'flash 0.5s'; 
    setTimeout(() => updateButton.style.animation = '', 500);
});

updateButton.addEventListener('mouseover', function() {
    updateButton.style.transform = 'scale(1.08)';
}); 
updateButton.addEventListener('mouseout', function() {
    updateButton.style.transform = 'scale(1)'; 
}); 
const style = document.createElement('style'); 
style.textContent = ` 
    @keyframes flash {
        0% { background-color: #007BFF; }
        50% { background-color: #FF8C00; }
        100% { background-color: #007BFF; } 
} `;
document.head.append(style);

##### Run [Cell 6](#cell-6) to export to the PARENT/HOME directory as .csv

## Cell 6

In [ ]:
df_val = pd.read_sql(sql_query.format(f"validity_{user_name_uscore}") , engine)
df_val.to_csv(os.path.join(os.path.expanduser("~"), f"{user_name}_exp_id_validation_output.csv"), index=False)
df_val